# Phân loại phân khúc đồng hồ bằng KNN và Random Forest

Trong notebook này, chúng ta sẽ:
1. Tải và khám phá dữ liệu
2. Tiền xử lý dữ liệu
3. Phân loại phân khúc đồng hồ sử dụng hai thuật toán:
   - K-Nearest Neighbors (KNN)
   - Random Forest (RF)
4. So sánh hiệu suất của hai thuật toán

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Bỏ qua các cảnh báo
import warnings
warnings.filterwarnings('ignore')

## 1. Tải và khám phá dữ liệu

Chúng ta sẽ tải dữ liệu từ file CSV và khám phá cấu trúc của dữ liệu.

In [ ]:
# Tải dữ liệu
# Kiểm tra cả train và test data để xem cấu trúc
train_df = pd.read_csv('../dataset/train_segmented.csv')
test_df = pd.read_csv('../dataset/test_segmented.csv')

print(f"Kích thước dữ liệu huấn luyện: {train_df.shape}")
print(f"Kích thước dữ liệu kiểm tra: {test_df.shape}")
print("\nDữ liệu huấn luyện:")
train_df.head()

In [ ]:
# Kiểm tra thông tin về các cột trong dữ liệu
train_df.info()

In [ ]:
# Thống kê mô tả
train_df.describe(include='all')

In [ ]:
# Kiểm tra giá trị null
print("Số lượng giá trị null trong tập huấn luyện:")
print(train_df.isnull().sum())

print("\nSố lượng giá trị null trong tập kiểm tra:")
print(test_df.isnull().sum())

In [ ]:
# Kiểm tra phân phối của biến mục tiêu (phân khúc đồng hồ)
plt.figure(figsize=(10, 6))
sns.countplot(x='price_segment', data=train_df)
plt.title('Phân phối các phân khúc đồng hồ')
plt.xlabel('Phân khúc')
plt.ylabel('Số lượng')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 2. Tiền xử lý dữ liệu

Trước khi áp dụng thuật toán, chúng ta cần tiền xử lý dữ liệu:
- Xử lý giá trị thiếu (nếu có)
- Mã hóa các biến phân loại
- Chuẩn hóa/Scaling các đặc trưng số

In [ ]:
# Xử lý dữ liệu bị thiếu (nếu có)
# Điền giá trị thiếu cho các cột số bằng giá trị trung bình
num_cols = train_df.select_dtypes(include=['int64', 'float64']).columns
for col in num_cols:
    if train_df[col].isnull().sum() > 0:
        train_df[col].fillna(train_df[col].mean(), inplace=True)
        test_df[col].fillna(train_df[col].mean(), inplace=True)

# Điền giá trị thiếu cho các cột phân loại bằng giá trị phổ biến nhất
cat_cols = train_df.select_dtypes(include=['object']).columns
for col in cat_cols:
    if train_df[col].isnull().sum() > 0:
        train_df[col].fillna(train_df[col].mode()[0], inplace=True)
        test_df[col].fillna(train_df[col].mode()[0], inplace=True)

# Kiểm tra lại sau khi xử lý
print("Số lượng giá trị null sau khi xử lý trong tập huấn luyện:")
print(train_df.isnull().sum().sum())

print("\nSố lượng giá trị null sau khi xử lý trong tập kiểm tra:")
print(test_df.isnull().sum().sum())

In [ ]:
# Chuẩn bị dữ liệu cho mô hình
# Tách đặc trưng và biến mục tiêu
X_train = train_df.drop('price_segment', axis=1)
y_train = train_df['price_segment']
X_test = test_df.drop('price_segment', axis=1)
y_test = test_df['price_segment']  # Lưu trữ nhãn thực tế từ tập kiểm tra

print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước X_test: {X_test.shape}")

# Kiểm tra các cột phân loại cần mã hóa
cat_cols = X_train.select_dtypes(include=['object']).columns
print(f"Các cột phân loại cần mã hóa: {list(cat_cols)}")

# Mã hóa các biến phân loại bằng Label Encoding hoặc One-Hot Encoding
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    label_encoders[col] = le
    
# Chuẩn hóa các đặc trưng số
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Chuyển đổi mảng trở lại DataFrame để dễ theo dõi
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("\nDữ liệu sau khi chuẩn hóa:")
X_train_scaled_df.head()

In [ ]:
# Mã hóa biến mục tiêu (nếu chưa là số)
if y_train.dtype == 'object':
    le_target = LabelEncoder()
    y_train_encoded = le_target.fit_transform(y_train)
    y_test_encoded = le_target.transform(y_test)
    
    # Hiển thị mapping của các phân khúc
    segment_mapping = dict(zip(le_target.classes_, range(len(le_target.classes_))))
    print("Mapping của các phân khúc:")
    for segment, code in segment_mapping.items():
        print(f"{segment}: {code}")
else:
    # Nếu đã là số, sử dụng trực tiếp
    y_train_encoded = y_train.values
    y_test_encoded = y_test.values
    print("Biến mục tiêu đã là số, không cần mã hóa thêm.")
    
    # Tạo nhãn lớp cho việc hiển thị
    unique_segments = sorted(y_train.unique())
    target_names = [str(segment) for segment in unique_segments]

## 3. Xây dựng và đánh giá mô hình

### 3.1. K-Nearest Neighbors (KNN)

Thuật toán KNN phân loại một điểm dữ liệu dựa trên phần lớn nhãn của k điểm dữ liệu gần nhất với nó. Chúng ta sẽ tìm giá trị k tối ưu bằng cross-validation.

In [ ]:
# Tìm giá trị k tốt nhất
k_range = list(range(1, 31))
k_scores = []

# Chia dữ liệu huấn luyện thành tập huấn luyện và tập validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train_encoded, test_size=0.2, random_state=42
)

# Tính điểm cho từng giá trị k
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_split, y_train_split)
    scores = knn.score(X_val_split, y_val_split)
    k_scores.append(scores)

# Vẽ đồ thị điểm số
plt.figure(figsize=(10, 6))
plt.plot(k_range, k_scores)
plt.xlabel('Giá trị của K')
plt.ylabel('Độ chính xác')
plt.title('Độ chính xác của KNN với các giá trị K khác nhau')
plt.grid(True)
plt.show()

# Tìm k tốt nhất
best_k = k_range[k_scores.index(max(k_scores))]
print(f"Giá trị k tốt nhất: {best_k} với độ chính xác: {max(k_scores):.4f}")

In [ ]:
# Huấn luyện mô hình KNN với k tốt nhất
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(X_train_scaled, y_train_encoded)

# Đánh giá trên tập huấn luyện
y_train_pred_knn = knn_best.predict(X_train_scaled)
print("Độ chính xác trên tập huấn luyện của KNN:", accuracy_score(y_train_encoded, y_train_pred_knn))

# Đánh giá bằng cross-validation
cv_scores_knn = cross_val_score(knn_best, X_train_scaled, y_train_encoded, cv=5)
print(f"Điểm cross-validation của KNN: {cv_scores_knn.mean():.4f} ± {cv_scores_knn.std():.4f}")

# Ma trận nhầm lẫn trên tập huấn luyện
plt.figure(figsize=(10, 8))
cm_knn = confusion_matrix(y_train_encoded, y_train_pred_knn)

# Xác định nhãn cho trục
if y_train.dtype == 'object':
    labels = le_target.classes_
else:
    labels = [str(i) for i in sorted(y_train.unique())]

sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.title('Ma trận nhầm lẫn của KNN trên tập huấn luyện')
plt.tight_layout()
plt.show()

# Báo cáo phân loại chi tiết
print("\nBáo cáo phân loại chi tiết của KNN:")
if y_train.dtype == 'object':
    print(classification_report(y_train_encoded, y_train_pred_knn, target_names=le_target.classes_))
else:
    print(classification_report(y_train_encoded, y_train_pred_knn, 
                              target_names=[str(i) for i in sorted(y_train.unique())]))

### 3.2. Random Forest (RF)

Random Forest là một thuật toán ensemble sử dụng nhiều cây quyết định để đưa ra dự đoán. Chúng ta sẽ tìm kiếm các tham số tốt nhất cho Random Forest.

In [ ]:
# Huấn luyện mô hình Random Forest với các tham số mặc định
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train_encoded)

# Đánh giá trên tập huấn luyện
y_train_pred_rf = rf.predict(X_train_scaled)
print("Độ chính xác trên tập huấn luyện của Random Forest:", 
      accuracy_score(y_train_encoded, y_train_pred_rf))

# Đánh giá bằng cross-validation
cv_scores_rf = cross_val_score(rf, X_train_scaled, y_train_encoded, cv=5)
print(f"Điểm cross-validation của Random Forest: {cv_scores_rf.mean():.4f} ± {cv_scores_rf.std():.4f}")

# Ma trận nhầm lẫn trên tập huấn luyện
plt.figure(figsize=(10, 8))
cm_rf = confusion_matrix(y_train_encoded, y_train_pred_rf)

# Xác định nhãn cho trục
if y_train.dtype == 'object':
    labels = le_target.classes_
else:
    labels = [str(i) for i in sorted(y_train.unique())]

sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.title('Ma trận nhầm lẫn của Random Forest trên tập huấn luyện')
plt.tight_layout()
plt.show()

# Báo cáo phân loại chi tiết
print("\nBáo cáo phân loại chi tiết của Random Forest:")
if y_train.dtype == 'object':
    print(classification_report(y_train_encoded, y_train_pred_rf, target_names=le_target.classes_))
else:
    print(classification_report(y_train_encoded, y_train_pred_rf, 
                              target_names=[str(i) for i in sorted(y_train.unique())]))

In [ ]:
# Tìm tham số tốt nhất cho Random Forest bằng GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Thực hiện tìm kiếm lưới
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), 
                         param_grid=param_grid, 
                         cv=5, 
                         n_jobs=-1,
                         verbose=1)

grid_search.fit(X_train_scaled, y_train_encoded)

# In ra tham số tốt nhất
print("Tham số tốt nhất cho Random Forest:")
print(grid_search.best_params_)
print(f"Điểm số tốt nhất: {grid_search.best_score_:.4f}")

In [ ]:
# Huấn luyện Random Forest với tham số tốt nhất
rf_best = grid_search.best_estimator_

# Đánh giá trên tập huấn luyện
y_train_pred_rf_best = rf_best.predict(X_train_scaled)
print("Độ chính xác trên tập huấn luyện của Random Forest tốt nhất:", 
      accuracy_score(y_train_encoded, y_train_pred_rf_best))

# Đánh giá bằng cross-validation
cv_scores_rf_best = cross_val_score(rf_best, X_train_scaled, y_train_encoded, cv=5)
print(f"Điểm cross-validation của Random Forest tốt nhất: {cv_scores_rf_best.mean():.4f} ± {cv_scores_rf_best.std():.4f}")

# Ma trận nhầm lẫn trên tập huấn luyện
plt.figure(figsize=(10, 8))
cm_rf_best = confusion_matrix(y_train_encoded, y_train_pred_rf_best)

# Xác định nhãn cho trục
if y_train.dtype == 'object':
    labels = le_target.classes_
else:
    labels = [str(i) for i in sorted(y_train.unique())]

sns.heatmap(cm_rf_best, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.title('Ma trận nhầm lẫn của Random Forest tốt nhất trên tập huấn luyện')
plt.tight_layout()
plt.show()

# Báo cáo phân loại chi tiết
print("\nBáo cáo phân loại chi tiết của Random Forest tốt nhất:")
if y_train.dtype == 'object':
    print(classification_report(y_train_encoded, y_train_pred_rf_best, target_names=le_target.classes_))
else:
    print(classification_report(y_train_encoded, y_train_pred_rf_best, 
                              target_names=[str(i) for i in sorted(y_train.unique())]))

In [ ]:
# Phân tích tầm quan trọng của đặc trưng trong Random Forest
feature_importances = rf_best.feature_importances_
feature_names = X_train.columns

# Sắp xếp các đặc trưng theo độ quan trọng
sorted_idx = feature_importances.argsort()[::-1]

# Vẽ biểu đồ cột
plt.figure(figsize=(12, 8))
plt.barh(range(len(sorted_idx)), feature_importances[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
plt.xlabel('Độ quan trọng')
plt.title('Độ quan trọng của các đặc trưng trong Random Forest')
plt.tight_layout()
plt.show()

## 4. So sánh hiệu suất của các mô hình

So sánh hiệu suất của hai mô hình: KNN và Random Forest

In [ ]:
# So sánh hiệu suất của các mô hình
models = ['KNN', 'Random Forest (Default)', 'Random Forest (Optimized)']
accuracy_train = [accuracy_score(y_train_encoded, y_train_pred_knn),
                  accuracy_score(y_train_encoded, y_train_pred_rf),
                  accuracy_score(y_train_encoded, y_train_pred_rf_best)]
cv_scores = [cv_scores_knn.mean(), cv_scores_rf.mean(), cv_scores_rf_best.mean()]

# Tạo biểu đồ so sánh
plt.figure(figsize=(12, 8))
x = np.arange(len(models))
width = 0.35

plt.bar(x - width/2, accuracy_train, width, label='Độ chính xác trên tập huấn luyện')
plt.bar(x + width/2, cv_scores, width, label='Độ chính xác Cross-validation')

plt.xlabel('Mô hình')
plt.ylabel('Độ chính xác')
plt.title('So sánh hiệu suất của các mô hình')
plt.xticks(x, models)
plt.ylim([0, 1])
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 5. Dự đoán trên tập kiểm tra

Sử dụng mô hình tốt nhất để dự đoán phân khúc đồng hồ trên tập kiểm tra.

In [ ]:
# Chọn mô hình tốt nhất dựa trên điểm cross-validation
if cv_scores_knn.mean() > cv_scores_rf_best.mean():
    best_model = knn_best
    model_name = "KNN"
else:
    best_model = rf_best
    model_name = "Random Forest"

print(f"Mô hình tốt nhất là: {model_name}")

# Dự đoán trên tập kiểm tra
y_test_pred = best_model.predict(X_test_scaled)

# Đánh giá trên tập kiểm tra
print(f"\nĐộ chính xác trên tập kiểm tra: {accuracy_score(y_test_encoded, y_test_pred):.4f}")

# Ma trận nhầm lẫn trên tập kiểm tra
plt.figure(figsize=(10, 8))
cm_test = confusion_matrix(y_test_encoded, y_test_pred)

# Xác định nhãn cho trục
if y_train.dtype == 'object':
    labels = le_target.classes_
else:
    labels = [str(i) for i in sorted(y_train.unique())]

sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.title(f'Ma trận nhầm lẫn của {model_name} trên tập kiểm tra')
plt.tight_layout()
plt.show()

# Báo cáo phân loại chi tiết
print("\nBáo cáo phân loại chi tiết trên tập kiểm tra:")
if y_train.dtype == 'object':
    print(classification_report(y_test_encoded, y_test_pred, target_names=le_target.classes_))
else:
    print(classification_report(y_test_encoded, y_test_pred, 
                               target_names=[str(i) for i in sorted(y_train.unique())]))

# Lưu kết quả dự đoán
if y_train.dtype == 'object':
    # Nếu biến mục tiêu là chuỗi, cần chuyển đổi ngược mã hóa
    predicted_segments = le_target.inverse_transform(y_test_pred)
else:
    # Nếu biến mục tiêu đã là số, không cần chuyển đổi
    predicted_segments = y_test_pred

results_df = pd.DataFrame({
    'id': test_df.index if 'id' not in test_df.columns else test_df['id'],
    'actual_segment': y_test,
    'predicted_segment': predicted_segments
})

print("\nMẫu kết quả dự đoán:")
print(results_df.head(10))

# Lưu kết quả dự đoán
results_df.to_csv('../dataset/predicted_segments.csv', index=False)
print("\nĐã lưu kết quả dự đoán vào file '../dataset/predicted_segments.csv'")

## 6. Kết luận

Trong notebook này, chúng ta đã thực hiện các bước sau:

1. Tải và khám phá dữ liệu phân khúc đồng hồ
2. Tiền xử lý dữ liệu bằng cách xử lý giá trị thiếu, mã hóa biến phân loại và chuẩn hóa đặc trưng
3. Xây dựng và đánh giá hai mô hình phân loại:
   - K-Nearest Neighbors (KNN) với k tối ưu
   - Random Forest với tham số mặc định và tham số tối ưu
4. So sánh hiệu suất của các mô hình và chọn mô hình tốt nhất
5. Sử dụng mô hình tốt nhất để dự đoán phân khúc đồng hồ trên tập kiểm tra

Kết quả cho thấy mô hình [KNN/Random Forest] có hiệu suất tốt nhất với độ chính xác cross-validation là [X]%.

In [ ]:
# So sánh hiệu suất của các mô hình trên tập kiểm tra
# Tính độ chính xác trên tập kiểm tra cho từng mô hình
y_test_pred_knn = knn_best.predict(X_test_scaled)
y_test_pred_rf = rf.predict(X_test_scaled)
y_test_pred_rf_best = rf_best.predict(X_test_scaled)

accuracy_test = [
    accuracy_score(y_test_encoded, y_test_pred_knn),
    accuracy_score(y_test_encoded, y_test_pred_rf),
    accuracy_score(y_test_encoded, y_test_pred_rf_best)
]

# Tạo biểu đồ so sánh trên tập kiểm tra
plt.figure(figsize=(12, 8))
x = np.arange(len(models))
width = 0.35

plt.bar(x - width/2, accuracy_train, width, label='Độ chính xác trên tập huấn luyện')
plt.bar(x + width/2, accuracy_test, width, label='Độ chính xác trên tập kiểm tra')

plt.xlabel('Mô hình')
plt.ylabel('Độ chính xác')
plt.title('So sánh hiệu suất của các mô hình trên tập huấn luyện và tập kiểm tra')
plt.xticks(x, models)
plt.ylim([0, 1])
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# In ra độ chính xác trên tập kiểm tra
print("Độ chính xác trên tập kiểm tra:")
for model_name, acc in zip(models, accuracy_test):
    print(f"{model_name}: {acc:.4f}")